In [1]:
import pandas as pd
import numpy as np

In [2]:
#read csv files in data
train = pd.read_csv('data/train.csv')
test= pd.read_csv('data/test.csv')

In [3]:
#convert data types where coluhmns have values yes and no to 1 and 0
train = train.replace({'yes': 1, 'no': 0})
test = test.replace({'yes': 1, 'no': 0})

In [4]:
#  Import the necessary libraries
from sklearn.preprocessing import LabelEncoder
#Create an instance of LabelEncoder
lbl = LabelEncoder()

In [5]:

#Fit the label encoder on the combined data of train and test datasets
lbl.fit(list(train['ecology'].values) + list(test['ecology'].values))

#Convert the 'ecology' column in the train dataset
train['ecology'] = lbl.transform(list(train['ecology'].values))
#Convert the 'ecology' column in the test dataset
test['ecology'] = lbl.transform(list(test['ecology'].values))

In [6]:
#do label encoding for prodcut type as well
lbl.fit(list(train['product_type'].values) + list(test['product_type'].values))
train['product_type'] = lbl.transform(list(train['product_type'].values))
test['product_type'] = lbl.transform(list(test['product_type'].values))

In [7]:
train.drop(columns=['sub_area'], inplace=True)
test.drop(columns=['sub_area'], inplace=True)

In [8]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PolynomialFeatures

In [9]:
X = train.drop(columns=['price_doc'])
y = train[['price_doc']]

In [10]:
#save row id in a dataframe
row_id = test['row ID']
#drop it from test now
test = test.drop(columns=['row ID'])

In [ ]:
from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler

# Assume X is your feature matrix and y is the target variable
scaler = StandardScaler()
X_std = scaler.fit_transform(X)

# Define the lasso regression model
lasso = Lasso(alpha=1.0)

# Fit the model
lasso.fit(X_std, y)


C:\Users\hamza\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.612e+19, tolerance: 8.652e+15
  model = cd_fast.enet_coordinate_descent(


Lasso()

In [11]:
import statsmodels.api as sm

# Add a constant column to the feature matrix
XSM = sm.add_constant(train.drop(columns=['price_doc']))

# Fit the ordinary least squares (OLS) model
model = sm.OLS(train['price_doc'], XSM).fit()

# Get the p-values of all columns
p_values = model.pvalues

# Sort the p-values in ascending order
sorted_p_values = p_values.sort_values()

# Show only the bottom 35 columns with the lowest p-values
bottom_35_p_values = sorted_p_values[:30]

# Print the bottom 35 p-values
# print(bottom_35_p_values)


In [12]:
# Use the bottom 35 features above to train the model
if 'const' in bottom_35_p_values:
    bottom_35_p_values = bottom_35_p_values.drop('const')
# Select the bottom 35 variables from the dataset
Xp = train[list(bottom_35_p_values.index)]
# Do the same for the test dataset as well
testp = test[list(bottom_35_p_values.index)]

In [17]:
from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler, PolynomialFeatures

# Assume X is your feature matrix and y is the target variable
scaler = StandardScaler()
X_std = scaler.fit_transform(Xp)
test2= scaler.transform(testp)
# Create polynomial features
poly = PolynomialFeatures(2,include_bias=False, interaction_only=True)
X_poly = poly.fit_transform(X_std)
X_test=poly.transform(test2)

# Define the lasso regression model
lasso = Lasso(alpha=1.0)

# Fit the model
lasso.fit(X_poly, y)


C:\Users\hamza\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.547e+19, tolerance: 8.652e+15
  model = cd_fast.enet_coordinate_descent(


Lasso()

In [18]:
price_doc = lasso.predict(X_test)

In [19]:
#Create a DataFrame for the predictions
predictions_df = pd.DataFrame(price_doc, columns=['price_doc'])

#Add the record ID from the test data to the predictions DataFrame
predictions_df.insert(0, 'row ID', row_id)

#Save the predictions to a CSV file
predictions_df.to_csv('lasso.csv', index=False)